In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup as bs
import psycopg2 as ps

# An Exercise in Creating an ETL (Extract, Transform, Load) Pipeline.

The International Weightlifting Federation (IWF) is the governing body for the sport of Olympic Weightlifting. They decide on and enforce the qualifications for Olympic qualification. They also put on international competitions in conjunction with the national governing body of Olympic Weightlifting in whatever country the competition is hosted in. 

I am taking data from the official IWF website (https://beta.iwf.sport/) and performing an exercise in creating an ETL pipeline. I will be using Python packages such as Tabula and bs4 to respectively read in pdf competition result books, and scrape the event results page of the IWF website to aggregate and organize international competition results.

After aggregation, I will be using TabPy (a Tableau Python server) to build dashboards visualizing various statistics and insights that can be learned from these competition results. 

I am a fan of the sport and have been personally training and competing for about eight years now. I thought this would be an interesting project to work on in order to learn some industry standard technologies and methodologies that would look good on a résumé. 

In the sport of Olympic Weightlifting, each athlete gets a total of six attempts in competition. Three [Snatch](https://www.youtube.com/watch?v=RqrBaArr1uI) (SN) attempts and three [Clean & Jerk](https://www.youtube.com/watch?v=kn73_fb_eOs) (CJ) attempts. The athlete's best successful lift in each discipline is added together to form their total. The athlete with the highest total in each weight class wins. 

The current weight classes are as follows:

    Men:        Women:
    55kg        45kg
    61kg        49kg 
    67kg        55kg
    73kg        59kg
    81kg        64kg
    89kg        71kg
    96kg        76kg
    102kg       81kg
    109kg       87kg
    +109kg      +87kg

# Collecting and Extracting the Data

In [2]:
def get_soup(url):
    """
    Given a URL, uses BeautifulSoup to create `soup` object which
    can be used for html scraping.

    Args:
        url (string) : url from IWF Results by Events page.
        
    Returns:
        soup (BeautifulSoup) : BeautifulSoup object.
    """

    page = requests.get(url)
    soup = bs(page.text, "html")

    return soup

In [3]:
def get_info(soup):
    """
    Given `soup`, scrapes page html to obtain competition information.

    Args:
        soup (BeautifulSoup) : BeautifulSoup object.
        
    Returns:
        comp_info (list) : List containing competition information on each
                           athlete, gotten from provided url.
    """
    
    comp_info_dirty = []
    grab_next = 0

    # Grabbing athlete information from all 4 pages of an event
    # (Men's Total, Women's Total, Men's Snatch/CJ, Women's Snatch/CJ)
    
    for line in soup.find_all("p"):
        if line.get_text().strip() == "Total:":
            grab_next = 1
        elif line.get_text().strip() == "Rank:":
            grab_next = 0
        elif grab_next == 1:
            comp_info_dirty.append(line.get_text().strip())

    # Removing labels from info list for easier entry into dfs

    comp_info = []
    # i, line in enumerate(comp_info_dirty)
    for line in comp_info_dirty:
        if ":" in line:
            line = line.split(":")[1].strip()
        comp_info.append(line)
    
    return comp_info

In [4]:
def get_names(soup):
    """
    Given soup, extracts athlete names.

    Args:
        soup (BeatifulSoup) : BeatifulSoup object.

    Returns:
        names_men (list) : List containing the names of each male athlete.
        names_women (list) : List contained the names of each female athlete.
    """

    name_soup_men = soup.find("div", {"id" : "men_total"})
    html_split_men = name_soup_men.get_text().strip().split("\n")
    elements_men = [i for i in html_split_men if i != ""]

    name_soup_women = soup.find("div", {"id" : "women_total"})
    html_split_women = name_soup_women.get_text().strip().split("\n")
    elements_women = [i for i in html_split_women if i != ""]
    
    names_men = []
    name_next = 0
    for i in elements_men:
        if "Rank: " in i:
            name_next = 1
        elif "kg Men" in i:
            names_men.append(i)
        elif name_next == 1:
            names_men.append(i)
            name_next = 0

    names_women = []
    name_next = 0
    for i in elements_women:
        if "Rank: " in i:
            name_next = 1
        elif "kg Women" in i:
            names_women.append(i)
        elif name_next == 1:
            names_women.append(i)
            name_next = 0
    
    return names_men, names_women

In [5]:
def get_athlete_numbers(soup, athlete_names_men, athlete_names_women):
    """
    Given a list of all athletes in a competition, find the 
    number of athletes in each session for both males and females.

    Args:
        soup (BeatifulSoup) : BeautifulSoup object.
        athlete_names_men (list) : List of male athlete names.
        athlete_names_women (list) : List of female athlete names.

    Returns:
        athletes_per_class (dict) : Dictionary containing weight classes for 
                                    each gender and the number of athletes in that 
                                    class as key:value pairs.
        total_male_athletes (int) : Number of male athletes in a given competition.
        total_female_athletes (int) : Number of female athletes in a given competition.
    """

    # Dictionary containing weight classes and the number of athletes
    # in that weight class as key:value pairs. Male weight classes 
    # are listed first.
    athletes_per_class = {}
    
    for class_ in soup.find_all("h3"):
        athletes_per_class[class_.get_text()] = 0

    class_ = ""
    for line in athlete_names_men:
        if "kg Men" in line:
            class_ = line
        else:
            athletes_per_class[class_] += 1


    class_ = ""
    for line in athlete_names_women:
        if "kg Women" in line:
            class_ = line
        else:
            athletes_per_class[class_] += 1

    total_male_athletes = sum([athletes_per_class[class_] for class_ in athletes_per_class if "Men" in class_])
    total_female_athletes = sum([athletes_per_class[class_] for class_ in athletes_per_class if "Women" in class_])
    
    return athletes_per_class, total_male_athletes, total_female_athletes

In [6]:
def make_dfs(soup, total_male_athletes, total_female_athletes, athletes_per_class, comp_info, comp_id):
    """
    Creates data frames of competition information for all male and female 
    weight classes.

    Args:
        soup (BeatifulSoup) : BeautifulSoup object.
        total_male_athletes (int) : Number of male athletes in a given competition.
        total_female_athletes (int) : Number of female athletes in a given competition.
        athletes_per_class (dict) : Dictionary containing weight classes for 
                                    each gender and the number of athletes in that 
                                    class as key:value pairs.
        comp_info (list) : List containing competition information on each athlete.
        comp_id (int) : Unique competition ID as found in event page URL.

     Returns :
         results_dict_unmarked (dict) : Dictionary containing competition results for all male 
                                        and female weight classes. Missed attempts are not 
                                        indicated.
    """
    results_dict_unmarked = {}

    for class_ in soup.find_all("h3"):
        results_dict_unmarked[class_.get_text()] = ...


    start_index = total_male_athletes * 9 + 1

    men_dfs_created = 0
    gender = "M"

    for class_ in athletes_per_class:

        # Result dfs for men have been created, move on to women
        if "Women" in class_ and men_dfs_created == 0:
            men_dfs_created = 1
            start_index += total_female_athletes * 9 
            gender = "F"
        
        stop_index_sn = start_index + athletes_per_class[class_]*10
    
        start_index_cj = stop_index_sn + 1
        stop_index_cj = start_index_cj + athletes_per_class[class_]*10
    
        start_index_total = stop_index_cj + 1
        stop_index_total = start_index_total + athletes_per_class[class_]*9

        sn_d = {
                "comp_id" : comp_id,
                "Name" : comp_info[start_index + 1:stop_index_sn:10],
                "Gender" : gender,
                "Nation" : comp_info[start_index + 2:stop_index_sn:10],
                "Born" : comp_info[start_index + 3:stop_index_sn:10],
                "Bodyweight" : comp_info[start_index + 4:stop_index_sn:10], 
                "Session" : comp_info[start_index + 5:stop_index_sn:10],
                "SN 1" : comp_info[start_index + 6:stop_index_sn:10],
                "SN 2" : comp_info[start_index + 7:stop_index_sn:10], 
                "SN 3" : comp_info[start_index + 8:stop_index_sn:10],
                "Best SN" : comp_info[start_index + 9:stop_index_sn:10],
                # "Total" : comp_info[start_index_total + 8:stop_index_total:9] 
                }

        cj_d = {
                "Name" : comp_info[start_index_cj + 1:stop_index_cj:10],
                "CJ 1" : comp_info[start_index_cj + 6:stop_index_cj:10],
                "CJ 2" : comp_info[start_index_cj + 7:stop_index_cj:10],
                "CJ 3" : comp_info[start_index_cj + 8:stop_index_cj:10],
                "Best CJ" : comp_info[start_index_cj + 9:stop_index_cj:10]
                }
        
        sn_df = pd.DataFrame(data=sn_d)
        cj_df = pd.DataFrame(data=cj_d)
        results_dict_unmarked[class_] = [sn_df, cj_df]
        start_index = stop_index_total + 1

    return results_dict_unmarked

In [7]:
def mark_missed_attempts(results_dict_unmarked, soup):
    """
    Iterates through SN and CJ result data frames in results_dict
    and marks missed attempts.

    Args: 
        results_dict_unmarked (dict) : Dictionary containing competition results for all male 
                                       and female weight classes. Missed attempts are not 
                                       indicated.
        soup (BeautifulSoup) : BeautifulSoup object.

    Returns:
        results_dict (dict) : Dictionary of competition results with missed attempts indicated. 
    """

    # Reading in male and female athletes' results in html 
    html_snatchjerk_men = soup.find("div", {"id" : "men_snatchjerk"})
    html_snatchjerk_women = soup.find("div", {"id" : "women_snatchjerk"})
    
    # Using regular expressions to extract missed attempts from html strings
    # Matching all instances of 1 or more digit (0-9)
    missed_attempts_pattern = re.compile(r'\d+|(\-{3})')
    
    # Applying missed_attempts_pattern
    html_tags_men = html_snatchjerk_men.find_all('strong', text=missed_attempts_pattern)
    html_tags_women = html_snatchjerk_women.find_all('strong', text=missed_attempts_pattern)

    # First (total_male_athletes * 3) indices of html_tags contain total results information
    start_index = 0 

    men_marked = 0

    for class_ in results_dict_unmarked:
        
        for index, df in enumerate(results_dict_unmarked[class_]):

            if "Women" in class_ and men_marked == 0:
                men_marked = 1
                start_index = 0
    
            stop_index = start_index + len(df) * 3
    
            if men_marked == 0:
                results = html_tags_men[start_index:stop_index]
                
            elif men_marked == 1:
                results = html_tags_women[start_index:stop_index]

            # Setting iterators to slice results list, updated after each athlete's 3 SN/CJ attempts
            # have been marked
            j = 0
            k = j + 3
            for athlete in df.index:

                # Marking SN attempts
                if index == 0:
                    
                    # Setting i = 1 to begin with 1st attempt in the SN
                    i = 1
                    
                    for attempt in results[j:k]:
                        if "<strike>" in str(attempt):
                            df.loc[athlete, 'SN' + ' ' + str(i)] = f'{df.loc[athlete, "SN" + " " + str(i)]}' + ' ' + 'x' 
                        elif "---" in str(attempt):
                            pass
                        
                        i += 1
                    
                # Marking CJ attempts
                elif index == 1:
                    i = 1
                    for attempt in results[j:k]:
                        if "<strike>" in str(attempt):
                            df.loc[athlete, 'CJ' + ' ' + str(i)] = f'{df.loc[athlete, "CJ" + " " + str(i)]}' + ' ' + 'x' 
                        elif "---" in str(attempt):
                            pass
    
                        i += 1

                # Moving to the next three attempts to be marked
                j = k
                k += 3

            # Updating 
            if index == 0:
                start_index = stop_index 
            elif index == 1:
                start_index = stop_index + len(df) * 2

    results_dict_marked = results_dict_unmarked

    return results_dict_marked

In [8]:
def merge_dfs(results_dict_marked):
    """
    Iterates through each weightclass in results_dict and merges
    SN/CJ result dataframes by athlete name. Result will be a singular
    dataframe for each weight class containing SN and CJ result information.

    Args:
        results_dict_marked (dict): Dictionary of competition results with missed attempts indicated. 
                                    SN/CJ results are contained at index 0/1 respectively.

    Returns: 
        results_dict_grouped (dict) : Dictionary of competition results. SN/CJ results displayed in a single
                                      dataframe.
    """
    
    results_dict_grouped = {}
    for class_ in results_dict_marked:
        sn_results = results_dict_marked[class_][0]
        cj_results = results_dict_marked[class_][1]
        results_dict_grouped[class_] = pd.merge(sn_results, cj_results, how="outer", on="Name")

        # Moving "Total" column to the right of df
        # total_col = results_dict_grouped[class_].pop("Total")
        # results_dict_grouped[class_].insert(13, "Total", total_col)
        
    return results_dict_grouped

In [9]:
def comp_data_pipeline(url, comp_id):
    """
    Calls all previously defined loading/cleaning functions.

    Args:
        url (str) : url from IWF Results by Events page.

    Returns:
        results_dict (dict) : Dictionary of competition results.
    
    """
    # Creating BeautifulSoup object for html scraping.
    soup = get_soup(url)

    # Extracting relevant competition information.
    comp_info = get_info(soup)

    # Extracting athlete names from comp_info list.
    athlete_names_men, athlete_names_women = get_names(soup)

    # Calculating relevant athlete number for data frame construction.
    athletes_per_class, total_male_athletes, total_female_athletes = get_athlete_numbers(soup, athlete_names_men, athlete_names_women)

    # Creating unmarked dictionary of competition results.
    results_dict_unmarked = make_dfs(soup, total_male_athletes, total_female_athletes, 
                                     athletes_per_class, comp_info, comp_id)

    # Marking missed attempts in results dictionary.
    results_dict_marked = mark_missed_attempts(results_dict_unmarked, soup)

    # Merge SN/CJ result dfs into a single df.
    results_dict = merge_dfs(results_dict_marked)

    return results_dict 

# Competition Logistic Information

I would like to also hold competition information such as the competition type, the location, and the date of the competition. To find this information, more html scraping of the IWF Results by Events page is required.

In [10]:
events_page_soup = get_soup("https://iwf.sport/results/results-by-events/?event_type=all&event_age=all&event_nation=all")

In [11]:
event_page_url = "https://iwf.sport/results/results-by-events/"

The previously defined data pipeline which gets competition results information into a pandas dataframe for each weight class requires a url as input. The url for each specific event page is found within <a> tags on the IWF Results by Events page.

In [12]:
def make_comp_logistics_df(url):
    """
    Given the IWF Results by Events url, creates a pandas dataframe containing
    the type, location, and date of each competition on the page.

    Args:
        url (str) : IWF Results by Events url.
    Returns:
        logisitcs_df (pandas data frame): Data frame containing compeition logistic information.
    """

    # Defining Results by Events Page BeatifulSoup object
    events_page_soup = get_soup(url)

    comp_id = []
    comp_names = []
    comp_locations = []
    comp_dates = []

    # Each competition has a unique ID, grabbing that here
    for line in events_page_soup.find_all("a", {"class" : "card"}):
        id = line.get('href').split("=")[1]
        comp_id.append(int(id))

    # On the Results by Events page, each competition title is contained within the class below
    for name in events_page_soup.find_all("div", {"class" : "col-md-5 col-12 not__cell__767__full"}):
        comp_names.append(name.get_text().strip())

    # The location of each competition is contained within the class below 
    for location in events_page_soup.find_all("div", {"class" : "col-md-3 col-4 not__cell__767"}): 
        comp_locations.append(location.get_text().strip())

    # The date of each compeition is contained within the class below
    for date in events_page_soup.find_all("div", {"class" : "col-md-2 col-4 not__cell__767"}): 
        comp_dates.append(date.get_text().strip())

    d = {
        "comp_id" : comp_id,
        "comp_name" : comp_names,
        "comp_location" : comp_locations,
        "comp_date" : comp_dates
        }

    logistics_df = pd.DataFrame(data=d)

    return logistics_df

In [13]:
comp_logistics_df = make_comp_logistics_df("https://iwf.sport/results/results-by-events/")

In [14]:
comp_logistics_df.head()

,comp_id,comp_name,comp_location,comp_date
0,596,Pan-American Championships,"Caracas, VEN","Feb 24, 2024"
1,595,Oceania Championships,"Auckland, AUS","Feb 21, 2024"
2,594,European Championships,"Sofia, BUL","Feb 12, 2024"
3,593,African Championships,"Ismailia, EGY","Feb 05, 2024"
4,592,Asian Championships,"Tashkent, UZB","Feb 03, 2024"


# Connecting to PostgreSQL Database

In [15]:
hostname = "localhost"
database = "iwfetl"
username = "connersparks"
port_id = 5432

conn = None
cur = None

In [16]:
def connect_to_db(hostname, database, username, port_id):
    """
    Connects to PostgreSQL database.
    """

    try:
        conn = ps.connect(
            host=hostname, 
            dbname=database, 
            user=username, 
            port=port_id)
    
    except Exception as error:
        print(error)
    
    else:
        print("Connected!")

    return conn

# finally:
#     if cur is not None:
#         cur.close()
#     if conn is not None:    
#         conn.close()

In [17]:
conn = connect_to_db(hostname, database, username, port_id)

Connected!


In [18]:
curr = conn.cursor()

# Loading Competition Logistics Table Into Database

In [19]:
def create_comp_logistics_table(curr):
    """
    Given a cursor object, runs SQL create table command to create
    a table containing competition logistics information.

    Args:
        curr (Psycopg2 Cursor) : Cursor object.
    """
    create_competition_table_command = ( """ CREATE TABLE IF NOT EXISTS comp_logistics (
                                        comp_id INTEGER PRIMARY KEY,
                                        comp_name TEXT NOT NULL,
                                        comp_location TEXT NOT NULL,
                                        comp_date DATE NOT NULL
                                        )""")

    curr.execute(create_competition_table_command)

In [20]:
def check_comp_logistics_exists(curr, comp_id):
    """
    Checks if given comp_id exists in the comp_logistics table.

    Args:
        curr (Psycopg2 Cursor) : Cursor object.
        comp_id (int) : Competition's unique key.

    Returns: True if competition is found, False otherwise.
    """

    # %s enables a parameter to be passed into SQL query, comp_id in this case
    query = ("""SELECT comp_id FROM comp_logistics WHERE comp_id = %s""") 

    curr.execute(query, (comp_id,))

    return curr.fetchone() is not None

In [21]:
def update_comp_logistics_table(curr, df):
    tmp_df = pd.DataFrame(columns=["comp_id", "comp_name", "comp_location", "comp_date"])
    
    for i, row in df.iterrows():
        # If competition already exists within our table, do nothing
        if check_comp_logistics_exists(curr, row["comp_id"]):
            continue
        # If competition does not exists within our table, append it to a temporary df
        else:
            tmp_df.loc[len(tmp_df.index)] = row
    
    return tmp_df

In [22]:
def insert_comp_logistics(curr, comp_id, comp_name, comp_location, comp_date):
    
    insert_command = ("""INSERT INTO comp_logistics (comp_id, comp_name, comp_location, comp_date)
                    VALUES(%s, %s, %s, %s);""")

    row_to_insert = (comp_id, comp_name, comp_location, comp_date)

    curr.execute(insert_command, row_to_insert)

In [23]:
def append_from_comp_logistics_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_comp_logistics(curr, row["comp_id"], row["comp_name"], row["comp_location"], row["comp_date"])

In [24]:
def comp_logistics_wrapper(curr, comp_logistics_df):
    """
    Calls all functions to create and insert into comp_logistics table.

    Args:
        curr (psycopg2 Cursor) : psycopg2 cursor object.
        comp_logistics_df (pandas df) : Dataframe of competition logistics from IWF
                                        Results by Events page.
    """

    create_comp_logistics_table(curr)

    new_comp_logistics_df = update_comp_logistics_table(curr, comp_logistics_df)

    append_from_comp_logistics_df_to_db(curr, new_comp_logistics_df)

# Loading Competition Data Table Into Database

In [53]:
def create_comp_data_table(curr):
    """
    Given a cursor object, runs SQL create table command to create
    a table containing competition logistics information.

    Args:
        curr (psycopg2 Cursor) : Cursor object.
    """

    create_competition_data_table_command = (""" CREATE TABLE IF NOT EXISTS comp_data (
                                            comp_id INTEGER NOT NULL,
                                            Name TEXT NOT NULL,
                                            Gender CHAR(1) NOT NULL,
                                            Nation TEXT NOT NULL,
                                            Born DATE NOT NULL,
                                            Bodyweight FLOAT NOT NULL,
                                            Session CHAR(1) NOT NULL,
                                            sn_1 VARCHAR(10) NOT NULL,
                                            sn_2 VARCHAR(10) NOT NULL,
                                            sn_3 VARCHAR(10) NOT NULL,
                                            best_sn VARCHAR(10) NOT NULL,
                                            cj_1 VARCHAR(10) NOT NULL,
                                            cj_2 VARCHAR(10) NOT NULL,
                                            cj_3 VARCHAR(10) NOT NULL,
                                            best_cj VARCHAR(10) NOT NULL
                                            )""")

    curr.execute(create_competition_data_table_command)

In [54]:
def check_comp_data_exists(curr, comp_id, name, nation, bweight, sn_1, sn_2, sn_3, best_sn, cj_1, cj_2, cj_3, best_cj):
    """
    Checks if given comp_id exists in the comp_logistics table.

    Args:
        curr (psycopg2 Cursor) : Cursor object.
        comp_id (int) : Competition's unique key.

    Returns: True if competition is found, False otherwise.
    """

    # %s enables a parameter to be passed into SQL query, comp_id in this case
    query = ("""SELECT comp_id FROM comp_data 
                WHERE comp_id = %s AND Name = %s AND Nation = %s AND Bodyweight = %s 
                AND sn_1 = %s AND sn_2 = %s AND sn_3 = %s AND best_sn = %s
                AND cj_1 = %s AND cj_2 = %s AND cj_3 = %s AND best_cj = %s""") 

    vars = (comp_id, name, nation, bweight, sn_1, sn_2, sn_3, best_sn, cj_1, cj_2, cj_3, best_cj)

    curr.execute(query, vars)

    return curr.fetchone() is not None

In [61]:
def update_comp_data_table(curr, df):

    tmp_df = pd.DataFrame(columns=["comp_id",
                                   "Name",
                                   "Gender",
                                   "Nation",
                                   "Born",
                                   "Bodyweight",
                                   "Session",
                                   "SN 1",
                                   "SN 2",
                                   "SN 3",
                                   "Best SN",
                                   "CJ 1",
                                   "CJ 2",
                                   "CJ 3",
                                   "Best CJ",
                                   "Total"
                                  ])

    for i, row in df.iterrows():
        # If competition already exists within our table, do nothing
        if check_comp_data_exists(curr, row["comp_id"], row["Name"], row["Nation"], row["Bodyweight"], row["SN 1"], row["SN 2"], row["SN 3"], row["Best SN"], row["CJ 1"], row["CJ 2"], row["CJ 3"], row["Best CJ"]):
            continue
        else:
            tmp_df.loc[len(tmp_df.index)] = row

    return tmp_df

In [62]:
def insert_comp_data(curr, comp_id, name, gender, nation, born, bodyweight, session, sn_1, sn_2, sn_3, best_sn, cj_1, cj_2, cj_3, best_cj):
    
    insert_command = ("""INSERT INTO comp_data (comp_id, name, gender, nation, born, bodyweight, session, 
                                                sn_1, sn_2, sn_3, best_sn, 
                                                cj_1, cj_2, cj_3, best_cj)              
                        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""")

    row_to_insert = (comp_id, name, gender, nation, born, bodyweight, session, sn_1, sn_2, sn_3, best_sn, cj_1, cj_2, cj_3, best_cj)

    curr.execute(insert_command, row_to_insert)

In [63]:
def append_from_comp_data_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_comp_data(curr, row["comp_id"], row["Name"], row["Gender"], row["Nation"], 
                         row["Born"], row["Bodyweight"], row["Session"],
                         row["SN 1"], row["SN 2"], row["SN 3"], row["Best SN"],
                         row["CJ 1"], row["CJ 2"], row["CJ 3"], row["Best CJ"])

In [64]:
def comp_data_wrapper(curr, results_dict):
    """
    Calls all functions to create and insert into comp_data table.

    Args:
        curr (psycopg2 Cursor) : psycopg2 cursor object.
        results_dict (dict) : Dictionary of results from a single competition.
    """

    create_comp_data_table(curr)

    # Iterating through each weight class and loading each into comp_data table
    for class_ in results_dict:
        new_comp_data_table = update_comp_data_table(curr, results_dict[class_])

        append_from_comp_data_df_to_db(curr, new_comp_data_table)

# Updating Pipeline

In [69]:
def pipeline():
    """
    Iterates through each competition on the IWF Results by Events page and 
    calls all previously defined cleaning/loading functions.
    """

    events_page_url = "https://iwf.sport/results/results-by-events/"
    events_page_soup = get_soup(events_page_url)
    
    # Connecting to iwf_etl database and creating cursor
    conn = connect_to_db(hostname, database, username, port_id)
    curr = conn.cursor()

    # Creating competition logistics df
    comp_logistitcs_df = make_comp_logistics_df(events_page_url)

    # Creating and populating comp_logistics table
    comp_logistics_wrapper(curr, comp_logistitcs_df)
    
    # Iterating through each competition on IWF Results by Events page,
    # passing through pipeline, and loading into comp_data table
    for comp in events_page_soup.find_all("a", {"class" : "card"}):
        comp_id_str = comp.get('href')
        comp_link = events_page_url + comp_id_str

        # Grabbing comp_id and casting to int
        comp_id_int = int(comp.get('href').split('=')[1])

        # Calling pipeline on each competition
        results_dict = comp_data_pipeline(comp_link, comp_id_int)
        print(f"Results for competition {comp_id_int} aggregated.")

        try:
            comp_data_wrapper(curr, results_dict)
            print(f"Competition {comp_id_int} loaded! \n")
        except Exception as error:
            print(f"Competition {comp_id_int} failed to load. Halting pipeline.")
            print(error)
            break
    
    conn.commit()
    print("Complete!")

In [70]:
pipeline()

Connected!
Results for competition 596 aggregated.
Competition 596 loaded! 
Results for competition 595 aggregated.
Competition 595 loaded! 
Results for competition 594 aggregated.
Competition 594 loaded! 
Results for competition 593 aggregated.
Competition 593 loaded! 
Results for competition 592 aggregated.
Competition 592 loaded! 
Results for competition 587 aggregated.
Competition 587 loaded! 
Results for competition 586 aggregated.
Competition 586 loaded! 
Results for competition 581 aggregated.
Competition 581 loaded! 
Results for competition 589 aggregated.
Competition 589 loaded! 
Results for competition 588 aggregated.
Competition 588 loaded! 
Results for competition 590 aggregated.
Competition 590 loaded! 
Results for competition 584 aggregated.
Competition 584 loaded! 
Results for competition 583 aggregated.
Competition 583 loaded! 
Results for competition 582 aggregated.
Competition 582 loaded! 
Results for competition 577 aggregated.
Competition 577 loaded! 
Results for co